# **UNICEF Indicator Data Warehouse** - SDMX REST Web Service

In [2]:
# import necessary libraries 
import requests

Connecting to the UNICEFs data warehouse using the SDMX standard

In [3]:
endpoint = "https://sdmx.data.unicef.org/ws/public/sdmxapi/rest"

### 1. step: select a dataflow

Dataflows contain a set of data within a common structure. So I use the API to get a list of all those dataflows first. 

This requires adding the following to the endpoint and providing the parameters for ```format```, ```detail``` and ```reference```. I chose to get the data in the **JSON format**, with **full detail** and **no references**. 

In [4]:
dataflow_endpoint = "/dataflow/all/all/latest"
dataflow_params = {
    "format": "sdmx-json",
    "detail": "full",
    "reference": "none",
}

In [12]:
# get the response and checking the status code 
dataflow_response = requests.get(url=endpoint+dataflow_endpoint, params=dataflow_params)
print(dataflow_response.status_code)

# converting the response into JSON format, printing the response 
# I arrived at the final printing format by following and trying out different keys  
dataflows = dataflow_response.json()['data']['dataflows']
for dflow in dataflows:
    # getting the "id" will be necessary for the next API call
    print(f"name: {dflow['name']} - id: {dflow['id']}")

200
name: Brazil Country Office - id: BRAZIL_CO
name: Brazil SELO - id: BRAZIL_CO_SELO
name: CAP 2030 - id: CAP2030
name: Coundown 2030 - id: CD2030
name: Countdown - coverage indicators - id: CDCOV
name: Countdown - demographic indicators - id: CDDEM
name: Countdown - drivers indicators - id: CDDRIVER
name: Countdown - coverage - equiplot data - id: CDEQUIPLOT
name: Countdown - tier 2 indicators - id: CDT2
name: CD2030 All - id: CONSOLIDATED
name: EAPRO Cross Sector Indicators - id: DF_EAPRO_CROSS_SECTOR
name: ECARO for TransMonEE - id: TRANSMONEE
name: SDG Country Global Dataflow - id: DF_SDG_GLC
name: SDG Harmonized Global Dataflow - id: DF_SDG_GLH
name: MENARO Education - id: EDU
name: UNICEF Middle East and North Africa - id: MENARO
name: Paraguay - id: PARAGUAY
name: Pakistan Survey Data - id: PAKISTAN_CO
name: Children’s Rights and Business Atlas - id: CRBA
name: UNICEF South Asia - id: ROSA
name: UNESCO UIS Education - id: UIS_EDU
name: Education: Students and Teachers - id: ED